<b>
<p>
<center>
<font size="6">
DATS 6202 Final: Predicting Poverty Rates with Demographic Data
</font>
</center>
</p>

<p>
<center>
<font size="4">
Author: Jehan Bugli
</font>
</center>
</p>

# Introduction

## Problem & Motivation

[Nearly 10% of the United States](https://www.whitehouse.gov/cea/written-materials/2024/09/10/the-2023-income-poverty-and-health-insurance-reports-strong-household-income-gains-lower-official-poverty-uninsured-rate-near-record-low/#:~:text=The%20official%20poverty%20rate%2C%20which,Figure%202%20shows%20these%20trends) lives in poverty, presenting a serious issue at odds with the nation's status as a global economic powerhouse.

Understanding root causes of poverty is a natural first step to crafting policy solutions to better address this issue; some of these causes may be well within reach for regulatory agencies, while knowledge of immutable factors can still help guide assistance more intelligently.

Factors influencing poverty are manifold, and I cannot model them all within the scope of this project. However, I think it presents a unique opportunity to highlight a subset of potential inputs and evaluate their effects. In particular, I'm interested in how demographics and local circumstances influence poverty rates.







## Dataset and Data Collection Methodology

My examination uses the [American Community Survey (ACS)](https://www.census.gov/programs-surveys/acs/about.html), an annual survey conducted by the Census Bureau covering a wide range of topics (including poverty). This source stood out for a few reasons:
- ACS data is collected on a far more granular level than any other source to my knowledge
- The ACS tracks data points that overlapped exceptionally well with potential poverty factors I was interested in examining
- Data was readily available via public API, leaving no concerns about accessibility or integrity beyond my own ability to utilize those resources



Armed with a clear direction, my next step was to actually gather the data! The Census API offers [multiple options](https://api.census.gov/data/2022/acs.html) with respect to the ACS; the 'detailed tables' endpoint, containing the information of interest, includes both 1-year and 5-year breakouts. These represent a tradeoff between recency and specificity; I ultimately chose to use the 5-year breakout so that I could drill down to the ['block' level, which is the smallest geographical unit that the Census provides](https://www.census.gov/newsroom/blogs/random-samplings/2011/07/what-are-census-blocks.html).

I initially entered this investigation planning to use ZIP codes; however, the [lack of defined boundaries](https://proximityone.com/tracts_zips.htm#:~:text=Census%20tracts%20are%20polygons%20and,points%20not%20even%20one%20line) and reduced dataset observations led me away from this approach.



To retain modularity and save time, especially while fine-tuning my models, I opted to split Census data grabs out into a separate ACS_Data_Grab.py file. This file took the following steps:
- Define all desired data points from the ACS (by code and a short string) in a dictionary
- Make an initial call to the Census API to retrieve all state codes / names, saving those in a separate dictionary
- Get all of the data for each state, splitting queries into two halves (observing an API limitation on fields per request) and subsequently consolidate entries for each census block
- Save to a .csv file

Once I had this pipeline built out, the next major milestone was figuring out which exact data points were both available and worth investigating. In certain cases, for instance, I found that certain fields weren't broken down to the block level (e.g. certain nativity fields).

Importantly, I did not want to include any variables that could be seen as reasonably direct proxies for poverty. For instance, a count of individuals receiving food stamps, SNAP, or public assistance is a product of poverty without practical predictive value from a policy-making perspective. Similarly, data points involving employment, salary, etc. are not useful model inputs for my purposes.

I settled on a few major categories:
- Total population (used for calculations, not as a formula input)
- Individuals below the poverty level (my pre-transformed target variable)
- Counts of individuals in granular age brackets, split by gender (I combine these for a gender-agnostic view)
- Count of individuals with hispanic ethnicity
- Count of individuals by race
- Count of individuals by their method of transportation to work
- Count of individuals by educational attainment
- Count of available housing units
- The median year in which buildings were constructed
- Count of individuals by veteran status
- Count of individuals by their home language being English alone
- Count of individuals by travel time to work (60-89 and 90+ categories are combined for one 60+ category)
- State code (a crude attempt to account for location-based discrepancies in the absence of an innately geographic model)


I converted most variables to reflect percentages of the total population, standardizing interpretation over different blocks with different population sizes. State codes were converted into a set of dummy variables using one-hot encoding; dimensionality concerns were fully negated by performance gains without overfitting.

# Notebook Configuration

## Google drive

In [1]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/Colab Notebooks/final/FA24_DATS6202_Final/'

# Get the absolute path of the shallow utilities folder
abspath_util_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning/'

# Get the absolute path of the shallow models folder
abspath_model_shallow = '/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/models/p2_shallow_learning/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Make directory for grid search results to use
directory = os.path.dirname(abspath_curr + '/result/GridSearchCV/')
if not os.path.exists(directory):
    os.makedirs(directory)

print(os.path.exists(directory))

True


## Warning

In [3]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

## Matplotlib

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

## TensorFlow

In [5]:
# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Random seed

In [6]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

# Data Preprocessing

In [7]:
# Change working directory to the absolute path of the shallow utilities folder
%cd $abspath_util_shallow

# Import the shallow utitilities
%run pmlm_utilities_shallow.ipynb

/content/drive/My Drive/Colab Notebooks/teaching/gwu/machine_learning_I/code/utilities/p2_shallow_learning


## Loading the data

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load all of the data
df = pd.read_csv(abspath_curr + '/data/census_data.csv', header=0)

In [9]:
# Filter dataframe rows where the total census block has less than a certain population
# At the risk of potentially making the rural/urban split less representative of the overall population, this is meant to remove any extreme outlier observations in the dataset

pop_cutoff = 500
pop_filter = df['total_pop'] >= pop_cutoff

print(f"Removing dataframe rows with less than {pop_cutoff} people: {len(df)-(pop_filter).sum()}/{len(df)} observations")
df = df[pop_filter]
print(len(df))

Removing dataframe rows with less than 500 people: 13154/242336 observations
229182


In [10]:
# Some of the Census API values for 'med_year_built' are provided as '-666666666'; these are imputed using the average of the given state and county

# Group year built by state and county values, calculating the average
grouped_year_means = df.groupby(['state', 'county'])['med_year_built'].transform(lambda x: x[x > 0].mean())

# Use the average value of a given state + county for all blocks where the median year built is less than 0
df['med_year_built_imputed'] = df['med_year_built'].where(df['med_year_built'] > 0, grouped_year_means)

# Filter out NaN values in 'med_year_built_imputed' (indicating that the whole block is missing the metric) and drop the original column
df = df.dropna(subset=['med_year_built_imputed']).drop(columns=['med_year_built'])

In [11]:
# Use one-hot encoding for states as a rough approximation for geography given that this model otherwise does not incorporate location; dimensionality increases are a potential concern under this approach, but this ultimately proved useful while avoiding overfitting
# The state of residence intuitively seems like an important factor given the significant disparities in outcomes: https://www.americanprogress.org/data-view/poverty-data/poverty-data-map-tool/
# Note that this uses state codes as provided in the data, not state abbreviations/names: https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html

df = pd.get_dummies(df, columns=['state'])
# df.drop(columns=['state'], inplace=True)

In [12]:
# Drop known identifiers
df.drop(columns=['NAME', 'county', 'tract', 'block group'], inplace=True)

In [13]:
# By default, a raw number of housing units is provided; I convert this to individuals per unit since that seems like a more useful metric of housing availability

# Calculate new metric
df['indivs_per_housing_unit'] = df['total_pop'] / df['housing_units']

# Drop original housing units field
df.drop(columns='housing_units', inplace=True)

In [14]:
# The data includes both 60-89 minute and 90_ minute travel times; these are combined into a single 60+ minute field as a percentage of all individuals

df['travel_time_60+'] = (df['travel_time_60_89'] + df['travel_time_90+'])/df['travel_time_total']
df.drop(columns=['travel_time_60_89', 'travel_time_90+', 'travel_time_total'], inplace=True)

In [15]:
# Define a function to combine male/female ages since those are separated in the dataset and intuitively seem like they wouldn't warrant the increased complexity
# This accepts the male column name and the dataframe, automatically getting the female equivalent and combining them

def combine_ages(col_m):

  # Get the implied name of the associated female age column
  col_f = col_m.replace('male', 'female')

  # Construct a new name for the combined metric
  col_new = col_m.replace('male', 'age')

  # Create a new column adding both male and female individuals within a given age group
  df[col_new] = df[col_m] + df[col_f]

  # Drop the original columns
  df.drop(columns=[col_m, col_f], inplace=True)

  # Returns nothing

In [16]:
# Import regex library to search for column name stype
import re

# Iterate through columns, searching for male age columns (with the given pattern) and combining male/female age groups
for col in df.columns:

  match = re.match(r'^male_\d+.*$', col)

  if match:
    combine_ages(col)
    print(f"{col} & {col.replace('male', 'female')} -> {col.replace('male', 'age')}")

male_0_4 & female_0_4 -> age_0_4
male_5_9 & female_5_9 -> age_5_9
male_10_14 & female_10_14 -> age_10_14
male_15_17 & female_15_17 -> age_15_17
male_18_19 & female_18_19 -> age_18_19
male_20 & female_20 -> age_20
male_21 & female_21 -> age_21
male_22_24 & female_22_24 -> age_22_24
male_25_29 & female_25_29 -> age_25_29
male_30_34 & female_30_34 -> age_30_34
male_35_39 & female_35_39 -> age_35_39
male_40_44 & female_40_44 -> age_40_44
male_45_49 & female_45_49 -> age_45_49
male_50_54 & female_50_54 -> age_50_54
male_55_59 & female_55_59 -> age_55_59
male_60_61 & female_60_61 -> age_60_61
male_62_64 & female_62_64 -> age_62_64
male_65_66 & female_65_66 -> age_65_66
male_67_69 & female_67_69 -> age_67_69
male_70_74 & female_70_74 -> age_70_74
male_75_79 & female_75_79 -> age_75_79
male_80_84 & female_80_84 -> age_80_84
male_85+ & female_85+ -> age_85+


In [17]:
# Define a function that converts a given column into a percentage based on a certain total (generally I'll use the grouping-specific totals if available because they don't always match the main total)

def convert_pct(col, col_total='total_pop', new_name = None):

  # Rename the column with the provided new name (or append '_pct' by default)
  if new_name is None:
      new_name = col + '_pct'

  df[col] = df[col] / df[col_total]
  df.rename(columns={col: new_name}, inplace=True)

# Define a function that iterates through columns to execute percentage conversions on a set of columns with a common prefix / predictable syntax

def convert_pct_batch(col_string):
  for col in df.columns:
    if col.startswith(col_string) and col != col_string + '_total':
      convert_pct(col, col_string + '_total')

  df.drop(columns=col_string + '_total', inplace=True)


In [18]:
# Execute batch conversions

for string in ['age', 'race', 'ethnicity', 'edu', 'trans', 'vet', 'home_lang']:

  convert_pct_batch(string)

In [19]:
# Convert poverty level separately (note that this replaces the existing column with a '_pct' suffix to denote the alteration)
convert_pct('below_poverty_level')

In [20]:
median_poverty = df['below_poverty_level_pct'].quantile(0.5)

In [21]:


# Convert the poverty level into a binary classification based on the average
# See table A-3, 2018-2022 (matching the 5-year period of the survey data): https://www.census.gov/data/tables/2024/demo/income-poverty/p60-283.html

df['poverty_level_category'] = pd.cut(df['below_poverty_level_pct'],
  bins=[-0.01, 0, .1138, 1],
  labels=['None', 'Under Average', 'Above Average'],
  right=False
)

# df['poverty_level_category'] = pd.cut(df['below_poverty_level_pct'],
#   bins=[0, 0.02, 0.04, 0.06, 0.08, 0.1, 1],
#   labels=['0-2%', '2-4%', '4-6%', '6-8%', '8-10%', '10%+'],
#   right=False
# )

df.drop(columns='below_poverty_level_pct', inplace=True)

In [22]:
# Remove total population since it's no longer needed for conversions (and is in itself not super valuable due to the nature of block group calculations)

df.drop(columns='total_pop', inplace=True)

Dataframe dimensions before removal:

In [23]:
pd.DataFrame([[df.shape[0], df.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,229181,107


In [24]:
df.dropna(inplace=True)

Dataframe dimensions after removal:

In [25]:
pd.DataFrame([[df.shape[0], df.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,228697,107


In [26]:
# Get the name of the target
target = 'poverty_level_category'
df[target].head()

,poverty_level_category
0,Under Average
1,Under Average
2,Under Average
3,Under Average
4,Under Average


In [27]:
from sklearn.preprocessing import LabelEncoder

# The LabelEncoder
le = LabelEncoder()

# Encode categorical target in the combined data
df[target] = le.fit_transform(df[target].astype(str))

In [28]:
# Print the first 5 rows of df
df[target].head()

,poverty_level_category
0,1
1,1
2,1
3,1
4,1


In [29]:
# Use the train_test_split functionality from sklearn to create training, validation, and testing data splits

df_train_and_val, df_test_raw = train_test_split(df,random_state=random_seed,test_size=0.2)

df_train_raw, df_val_raw = train_test_split(df_train_and_val,random_state=random_seed,test_size=0.2)

# Make a copy of raw dataframes
df_train = df_train_raw.copy(deep=True)
df_val = df_val_raw.copy(deep=True)
df_test = df_test_raw.copy(deep=True)

In [30]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,146365,107


In [31]:
# Print the dimension of df_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,45740,107


## Encode the categorical target

## Splitting the feature and target

In [32]:
# Get the feature matrix
X_train = df_train[np.setdiff1d(df_train.columns, [target])].values
X_val = df_val[np.setdiff1d(df_val.columns, [target])].values
X_test = df_test[np.setdiff1d(df_test.columns, [target])].values

# Get the target vector
y_train = df_train[target].values
y_val = df_val[target].values
y_test = df_test[target].values

## Scaling the data

### Standardization
The code below shows how to standardize the data.

In [33]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#mms = MinMaxScaler()

# The StandardScaler
ss = StandardScaler()

#### Standardizing the features
The code below shows how to standardize the features.

In [34]:
# Standardize the training data
X_train = ss.fit_transform(X_train)

# Standardize the validation data
X_val = ss.transform(X_val)

# Standardize the test data
X_test = ss.transform(X_test)

In [35]:
# from imblearn.under_sampling import RandomUnderSampler

# rus = RandomUnderSampler(random_state=random_seed)

# X_train, y_train = rus.fit_resample(X_train, y_train)

# Decision Tree

## The Sklearn model

The code below shows how to train the sklearn decision tree model (on the training data) and use it for prediction (on the validation data).

In [36]:
# from sklearn.tree import DecisionTreeClassifier

# # The DecisionTreeClassifier
# dtc = DecisionTreeClassifier(class_weight='balanced', random_state=random_seed)

# # Train the decision tree classifier on the training data
# dtc.fit(X_train, y_train)

# # Get the prediction on the validation data
# y_val_pred = dtc.predict(X_val)

The code below shows how to get the precision, recall and f-score.

In [37]:
# from sklearn.metrics import precision_recall_fscore_support

# # Get the precision, recall, f-score and support
# precision, recall, fscore, support = precision_recall_fscore_support(y_val, y_val_pred, average='micro')

# # Print the precision, recall and f-score
# pd.DataFrame([[precision, recall, fscore]], columns=['precision', 'recall', 'f-score'])

## Plot the decision tree

## Plot the feature importance

The code below shows how to get the feature importance detected by decision tree.

In [38]:
# # Get the dataframe of feature and importance
# df_fi_dtc = pd.DataFrame(np.hstack((np.setdiff1d(df.columns, [target]).reshape(-1, 1), dtc.feature_importances_.reshape(-1, 1))),
#                          columns=['Features', 'Importance'])

# # Sort df_fi_dtc in descending order of the importance
# df_fi_dtc = df_fi_dtc.sort_values(ascending=False, by='Importance').reset_index(drop=True)

# # Print df_fi_dtc
# df_fi_dtc

The code below shows how to create the bar plot of feature importance.

In [39]:
# # Create a figure
# fig = plt.figure(figsize=(40, 20))

# # The bar plot of feature importance
# plt.bar(df_fi_dtc['Features'], df_fi_dtc['Importance'], color='green')

# # Set x-axis
# plt.xlabel('Features')

# # Set y-axis
# plt.ylabel('Importance')

# # Rotate the labels
# plt.xticks(rotation=90)

# # Save and show the figure
# plt.tight_layout()
# plt.savefig(abspath_curr + '/result/feature_importance_dtc.pdf')
# plt.show()

# Bagging

## Random Forest

### The Sklearn model

The code below shows how to train the sklearn random forest model (on the training data) and use it for prediction (on the validation data).

In [40]:
# from sklearn.ensemble import RandomForestClassifier

# # The RandomForestClassifier
# rfc = RandomForestClassifier(class_weight='balanced', random_state=random_seed)

# # Train the random forest classifier on the training data
# rfc.fit(X_train, y_train)

# # Get the prediction on the validation data
# y_val_pred = rfc.predict(X_val)

In [41]:
# # Get the precision, recall, f-score and support
# precision, recall, fscore, support = precision_recall_fscore_support(y_val, y_val_pred, average='micro')

# # Print the precision, recall and f-score
# pd.DataFrame([[precision, recall, fscore]], columns=['precision', 'recall', 'f-score'])

### Plot the feature importance

The code below shows how to get the feature importance detected by decision tree.

In [42]:
# # Get the dataframe of feature and importance
# df_fi_rfc = pd.DataFrame(np.hstack((np.setdiff1d(df.columns, [target]).reshape(-1, 1), rfc.feature_importances_.reshape(-1, 1))),
#                          columns=['Features', 'Importance'])

# # Sort df_fi_rfc in descending order of the importance
# df_fi_rfc = df_fi_rfc.sort_values(ascending=False, by='Importance').reset_index(drop=True)

# # Print df_fi_rfcs
# df_fi_rfc

The code below shows how to create the bar plot of feature importance.

In [43]:
# # Create a figure
# fig = plt.figure(figsize=(40, 20))

# # The bar plot of feature importance
# plt.bar(df_fi_rfc['Features'], df_fi_rfc['Importance'], color='green')

# # Set x-axis
# plt.xlabel('Features')

# # Set y-axis
# plt.ylabel('Importance')

# # Rotate the labels
# plt.xticks(rotation=90)

# # Save and show the figure
# plt.tight_layout()
# plt.savefig(abspath_curr + '/result/feature_importance_rfc.pdf')
# plt.show()

# Boosting

## Gradient Boosting

### The Sklearn model

The code below shows how to train the sklearn histogram-based gradient boosting classifier (on the training data) and use it for prediction (on the validation data).

In [44]:
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingClassifier

# # The HistGradientBoostingClassifier
# hgbc = HistGradientBoostingClassifier(random_state=random_seed)

# # Train the histogram-based gradient boosting classifier on the training data
# hgbc.fit(X_train, y_train)

# # Get the prediction on the validation data
# y_val_pred = hgbc.predict(X_val)

The code below shows how to get the precision, recall and f-score.

In [45]:
# # Get the precision, recall, f-score and support
# precision, recall, fscore, support = precision_recall_fscore_support(y_val, y_val_pred, average='micro')

# # Print the precision, recall and f-score
# pd.DataFrame([[precision, recall, fscore]], columns=['precision', 'recall', 'f-score'])

# Hyperparameter Tuning

## Creating the dictionary of the models
- In the dictionary:
    - the key is the acronym of the model
    - the value is the model

In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

models = {
  'dtc': DecisionTreeClassifier(class_weight='balanced', random_state=random_seed),
  'rfc': RandomForestClassifier(class_weight='balanced', random_state=random_seed),
  'hgbc': HistGradientBoostingClassifier(random_state=random_seed),
  'mlpc': MLPClassifier(early_stopping=True, random_state=random_seed)
}

## Creating the dictionary of the pipelines
In the dictionary:
- the key is the acronym of the model
- the value is the pipeline, which, for now, only includes the model

In [47]:
from sklearn.pipeline import Pipeline

pipes = {}

for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

## Getting the predefined split cross-validator

In [48]:
# Get the:
# feature matrix and target velctor in the combined training and validation data
# target vector in the combined training and validation data
# PredefinedSplit
# See the implementation in pmlm_utilities.ipynb
X_train_val, y_train_val, ps = get_train_val_ps(X_train, y_train, X_val, y_val)

## GridSearchCV

### Creating the dictionary of the parameter grids
- In the dictionary:
    - the key is the acronym of the model
    - the value is the parameter grid of the model

In [49]:
param_grids = {}

#### The parameter grid for decision tree
- The hyperparameters we want to fine-tune are:
    - min_samples_split
    - min_samples_leaf

- See details of the meaning of the hyperparametes in [sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

In [50]:
param_grids['dtc'] = [{
  'model__min_samples_split': [10],
  'model__min_samples_leaf': [20],
  'model__max_depth': [4],
  'model__max_features': ['sqrt']
}]

#### The parameter grid for random forest
- The hyperparameters we want to fine-tune are:
    - min_samples_split
    - min_samples_leaf

- See details of the meaning of the hyperparametes in [sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [51]:
# Update param_grids
param_grids['rfc'] = [{
  'model__min_samples_split': [20],
  'model__min_samples_leaf': [20],
  'model__n_estimators': [100],
  'model__max_depth': [15],
  'model__max_features': ['log2']
}]

#### The parameter grid for histogram-based gradient boosting
- The hyperparameters we want to fine-tune are:
- learning_rate
- min_samples_leaf

- See details of the meaning of the hyperparametes in [sklearn.ensemble.HistGradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html)

In [52]:

# Update param_grids
param_grids['hgbc'] = [{
  'model__learning_rate': [1],
  'model__min_samples_leaf': [20],
  'model__max_iter': [100],
  'model__max_depth': [3],
  'model__max_features': [0.5]
}]

#### The parameter grid for MLPClassifier
- The hyperparameters we want to fine-tune are:
    - alpha
    - learning_rate_init

- See details of the meaning of the hyperparametes in [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [53]:
# Update param_grids
param_grids['mlpc'] = [{
  'model__alpha': [10 ** -3],
  'model__learning_rate': ['adaptive', 'invscaling'],
  'model__early_stopping': [True],
  'model__hidden_layer_sizes': [(100,)]
}]

### Tuning the hyperparameters
Filler

In [54]:
from sklearn.model_selection import GridSearchCV

# The list of [best_score_, best_params_, best_estimator_] obtained by GridSearchCV
best_score_params_estimator_gs = []

# For each model
for acronym in pipes.keys():
    print(f"Processing: {acronym}")
    # GridSearchCV
    gs = GridSearchCV(estimator=pipes[acronym],
                      param_grid=param_grids[acronym],
                      scoring='f1_macro',
                      n_jobs=2,
                      cv=ps,
                      return_train_score=True)

    # Fit the pipeline
    gs = gs.fit(X_train_val, y_train_val)

    # Update best_score_params_estimator_gs
    best_score_params_estimator_gs.append([gs.best_score_, gs.best_params_, gs.best_estimator_])

    # Sort cv_results in ascending order of 'rank_test_score' and 'std_test_score'
    cv_results = pd.DataFrame.from_dict(gs.cv_results_).sort_values(by=['rank_test_score', 'std_test_score'])

    # Get the important columns in cv_results
    important_columns = ['rank_test_score',
                         'mean_test_score',
                         'std_test_score',
                         'mean_train_score',
                         'std_train_score',
                         'mean_fit_time',
                         'std_fit_time',
                         'mean_score_time',
                         'std_score_time']

    # Move the important columns ahead
    cv_results = cv_results[important_columns + sorted(list(set(cv_results.columns) - set(important_columns)))]

    # Write cv_results file
    cv_results.to_csv(path_or_buf=abspath_curr + 'result/GridSearchCV/' + acronym + '.csv', index=False)

# Sort best_score_params_estimator_gs in descending order of the best_score_
best_score_params_estimator_gs = sorted(best_score_params_estimator_gs, key=lambda x : x[0], reverse=True)

# Print best_score_params_estimator_gs
pd.DataFrame(best_score_params_estimator_gs, columns=['best_score', 'best_param', 'best_estimator'])

Processing: dtc
Processing: rfc
Processing: hgbc
Processing: mlpc


,best_score,best_param,best_estimator
0,0.648889,"{'model__max_depth': 15, 'model__max_features'...","((DecisionTreeClassifier(max_depth=15, max_fea..."
1,0.631728,"{'model__alpha': 0.001, 'model__early_stopping...","(MLPClassifier(alpha=0.001, early_stopping=Tru..."
2,0.600799,"{'model__learning_rate': 1, 'model__max_depth'...",(HistGradientBoostingClassifier(learning_rate=...
3,0.492304,"{'model__max_depth': 4, 'model__max_features':...",(DecisionTreeClassifier(class_weight='balanced...


Fine but now let's make the decent models vote

In [55]:
from sklearn.ensemble import VotingClassifier


models_optimal = {
  'rfc': RandomForestClassifier(class_weight='balanced', random_state=random_seed, min_samples_split=20, min_samples_leaf=20, n_estimators=100, max_depth=15, max_features='log2'),
  'hgbc': HistGradientBoostingClassifier(random_state=random_seed, learning_rate=1, min_samples_leaf=20, max_iter=100, max_depth=3, max_features=0.5),
  'mlpc': MLPClassifier(early_stopping=True, random_state=random_seed, alpha=10 ** -3, learning_rate='adaptive')
}

# Create the voting classifier
voting_clf = VotingClassifier(
    estimators=[('rfc', models_optimal['rfc']), ('hgbc', models_optimal['hgbc']), ('mlpc', models_optimal['mlpc'])],
    voting='hard'  # Use 'soft' for probability-based voting
)

# Train the voting classifier
voting_clf.fit(X_train, y_train)

# Make predictions
y_val_pred = voting_clf.predict(X_val)

# Evaluate performance (use appropriate metrics for multiclass classification)
from sklearn.metrics import classification_report, f1_score

print(classification_report(y_val, y_val_pred))
print("F1-score (macro):", f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

           0       0.41      0.25      0.31       722
           1       0.99      0.99      0.99     35870

    accuracy                           0.98     36592
   macro avg       0.70      0.62      0.65     36592
weighted avg       0.97      0.98      0.98     36592

F1-score (macro): 0.6510865321272316


# Model Selection
Here we will select best_estimator_gs as the best model. Later we will use this best model to generate the submission file for this kaggle competition.